<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Подготовка-данных" data-toc-modified-id="Подготовка-данных-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Подготовка данных</a></span></li><li><span><a href="#Исследование-задачи" data-toc-modified-id="Исследование-задачи-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Исследование задачи</a></span></li><li><span><a href="#Борьба-с-дисбалансом" data-toc-modified-id="Борьба-с-дисбалансом-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Борьба с дисбалансом</a></span></li><li><span><a href="#Тестирование-модели" data-toc-modified-id="Тестирование-модели-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Тестирование модели</a></span></li><li><span><a href="#Чек-лист-готовности-проекта" data-toc-modified-id="Чек-лист-готовности-проекта-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Чек-лист готовности проекта</a></span></li></ul></div>

# Отток клиентов

Из «Бета-Банка» стали уходить клиенты. Каждый месяц. Немного, но заметно. Банковские маркетологи посчитали: сохранять текущих клиентов дешевле, чем привлекать новых.

Нужно спрогнозировать, уйдёт клиент из банка в ближайшее время или нет. Вам предоставлены исторические данные о поведении клиентов и расторжении договоров с банком. 

Постройте модель с предельно большим значением *F1*-меры. Чтобы сдать проект успешно, нужно довести метрику до 0.59. Проверьте *F1*-меру на тестовой выборке самостоятельно.

Дополнительно измеряйте *AUC-ROC*, сравнивайте её значение с *F1*-мерой.

Источник данных: [https://www.kaggle.com/barelydedicated/bank-customer-churn-modeling](https://www.kaggle.com/barelydedicated/bank-customer-churn-modeling)

## Подготовка данных

<b>*Описание данных:*</b>

 <b>Признаки</b>

RowNumber — индекс строки в данных

CustomerId — уникальный идентификатор клиента

Surname — фамилия

CreditScore — кредитный рейтинг

Geography — страна проживания

Gender — пол

Age — возраст

Tenure — сколько лет человек является клиентом банка

Balance — баланс на счёте

NumOfProducts — количество продуктов банка, используемых клиентом

HasCrCard — наличие кредитной карты

IsActiveMember — активность клиента

EstimatedSalary — предполагаемая зарплата

 <b>Целевой признак</b>

Exited — факт ухода клиента

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split

from sklearn.metrics import f1_score
from sklearn.metrics import roc_auc_score

from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier

from sklearn.utils import shuffle


Осуществим краткий обзор данных

In [2]:
data_temp = pd.read_csv('/datasets/Churn.csv')
data_temp.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2.0,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1.0,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8.0,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1.0,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2.0,125510.82,1,1,1,79084.10,0


In [3]:
data_temp.tail()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
9995,9996,15606229,Obijiaku,771,France,Male,39,5.0,0.00,2,1,0,96270.64,0
9996,9997,15569892,Johnstone,516,France,Male,35,10.0,57369.61,1,1,1,101699.77,0
9997,9998,15584532,Liu,709,France,Female,36,7.0,0.00,1,0,1,42085.58,1
9998,9999,15682355,Sabbatini,772,Germany,Male,42,3.0,75075.31,2,1,0,92888.52,1
9999,10000,15628319,Walker,792,France,Female,28,NaN,130142.79,1,1,0,38190.78,0


In [4]:
data_temp.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   RowNumber        10000 non-null  int64  
 1   CustomerId       10000 non-null  int64  
 2   Surname          10000 non-null  object 
 3   CreditScore      10000 non-null  int64  
 4   Geography        10000 non-null  object 
 5   Gender           10000 non-null  object 
 6   Age              10000 non-null  int64  
 7   Tenure           9091 non-null   float64
 8   Balance          10000 non-null  float64
 9   NumOfProducts    10000 non-null  int64  
 10  HasCrCard        10000 non-null  int64  
 11  IsActiveMember   10000 non-null  int64  
 12  EstimatedSalary  10000 non-null  float64
 13  Exited           10000 non-null  int64  
dtypes: float64(3), int64(8), object(3)
memory usage: 1.1+ MB


В столбце Tenure (сколько лет человек является клиентом банка) имеются пропуски, которые могут быть связаны, в том числе с технической ошибкой в данных при их сборе. При этом наличие пустых значений может повлиять на результаты дальнейшей работы с данными.

Заменим пустые значения на 0. Удаление строк с пустыми значения в рассматриваемом случае считаю менее предподчительным, поскольку количество таких значений достаточно большое.

In [5]:
data_temp.loc[data_temp['Tenure'].isna()] = 0

In [6]:
data_temp.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   RowNumber        10000 non-null  int64  
 1   CustomerId       10000 non-null  int64  
 2   Surname          10000 non-null  object 
 3   CreditScore      10000 non-null  int64  
 4   Geography        10000 non-null  object 
 5   Gender           10000 non-null  object 
 6   Age              10000 non-null  int64  
 7   Tenure           10000 non-null  float64
 8   Balance          10000 non-null  float64
 9   NumOfProducts    10000 non-null  int64  
 10  HasCrCard        10000 non-null  int64  
 11  IsActiveMember   10000 non-null  int64  
 12  EstimatedSalary  10000 non-null  float64
 13  Exited           10000 non-null  int64  
dtypes: float64(3), int64(8), object(3)
memory usage: 1.1+ MB


Пустые значения исключены.

Далее целесообразно исключить столбцы RowNumber, CustomerId и Surname из данных, поскольку данные по таким столбцам не имеют значения для обучения и эффективности моделей.

In [7]:
data_temp = data_temp.drop(columns=['RowNumber', 'CustomerId', 'Surname'])

Далее необходимо осуществить обработку категориальных признаков техникой One-Hot Encoding для осуществления прямого кодирования данных.

In [8]:
data = pd.get_dummies(data_temp, drop_first=True)
data.shape

(10000, 14)

Обработка OHE осуществлена успешно.

Далее проведем разделение данных на тестовую, валидационную и тренировочную в два этапа.

In [9]:
features = data.drop(['Exited'], axis=1)
target = data['Exited']
features_train_valid, features_test, target_train_valid, target_test = train_test_split(features, target, test_size=0.2, random_state=12345, stratify=target)
features_train, features_valid, target_train, target_valid = train_test_split(features_train_valid, target_train_valid, test_size=0.2, random_state=12345, stratify=target_train_valid)
print(features_valid.shape, target_valid.shape)
print(features_test.shape, target_test.shape)
print(features_train.shape, target_train.shape)

(1600, 13) (1600,)
(2000, 13) (2000,)
(6400, 13) (6400,)


Разделение данных осуществлено успешно.

Данные подготовлены для обучения моделей и дальнейшей работы с ними.

**Вывод:**

В ходе обзора и подготовки данных установлено, что данные содержат пропуски (устранено) и избыточные столбцы с данными (исключены).

## Исследование задачи

Осуществим проверку баланса классов

In [10]:
data['Exited'].value_counts()

0    8146
1    1854
Name: Exited, dtype: int64

Таким образом, учитывая, что соотношение положительных и отрицательных классов близко не равно 1:1 (соотношение 4:1), можно прийти к выводу что в данных присуствует дисбаланс. Необходимо учитывая указанный факт при дальнейшем обучении модели.

Исследуем эффективность двух видов моделей: Решающее дерево и Случайный лес.

**Случайный лес:**

In [11]:
#вариант ДО изменений:

#for est in range(50, 130, 10):
#    model = RandomForestClassifier(n_estimators=est, random_state=12345, max_depth=10)
#    model.fit(features_train, target_train)
          
#    predicted_valid = model.predict(features_valid)
#    f1 = f1_score(target_valid, predicted_valid)
#    print("Количество деревьев:", est, "\n", "f1 метрика:", f1, "\n")

#вариант ПОСЛЕ изменений:
f1 = 0
for est in range(50, 130, 10):
    model = RandomForestClassifier(n_estimators=est, random_state=12345, max_depth=10)
    model.fit(features_train, target_train)
          
    predicted_valid = model.predict(features_valid)
    f1_temp = f1_score(target_valid, predicted_valid)
    if f1_temp > f1:
        f1 = f1_temp
        best_est = est
print("Лучшее количество деревьев:", est, "\n", "лучшая f1 метрика:", f1, "\n")



Лучшее количество деревьев: 120 
 лучшая f1 метрика: 0.5580357142857143 



In [12]:
prob_valid = model.predict_proba(features_valid)
prob_one_valid = prob_valid[:, 1]
auc_roc = roc_auc_score(target_valid, prob_one_valid)
print(auc_roc)

0.8593068055846259


Таким образом, экспериментально установлено, что оптимальным набором гиперпараметров при использовании "Случайного леса" является: количество деревьев - 120, глубина - 10. При этом учтена скорость совершения операции.

f1 - 0.558

roc_auc_score - 0.859

**Решающее дерево:**

In [13]:
f1_2 = 0
for depth in range(2, 15):
    model = DecisionTreeClassifier(random_state=12345, max_depth=depth)
    model.fit(features_train, target_train)
    predict_valid = model.predict(features_valid)
    f1_2_temp = f1_score(target_valid, predict_valid)
    if f1_2_temp > f1_2:
        f1_2 = f1_2_temp
        best_depth = depth
print("Лучшая глубина дерева", best_depth, ":",  "\n", " Лучшая f1 метрика:", f1_2, "\n")

Лучшая глубина дерева 7 : 
  Лучшая f1 метрика: 0.5642105263157894 



In [14]:

model = DecisionTreeClassifier(random_state=12345, max_depth=best_depth)
model.fit(features_train, target_train)

prob_valid1 = model.predict_proba(features_valid)
prob_one_valid1 = prob_valid1[:, 1]
auc_roc1 = roc_auc_score(target_valid, prob_one_valid1)
print(auc_roc1)

0.8143664839750795


Таким образом, экспериментально установлено, что оптимальным набором гиперпараметров при использовании "Решающего дерева" является: глубина - 7.

f1 - 0.564

roc_auc_score - 0.814

**Вывод:**

Учитывая вышеизложенные расчеты, установлено, что модель "Случайный лес" достигает наибольших показателей метрик F1 и roc_auc_score, в связи с чем, такая модель подлежит дальнейшему использованию.

## Борьба с дисбалансом

В целях исключения дисбаланса целесообразно попробовать применить атрибут class_weight при обучении моделей "Случайный лес" и "Решающее дерево".

In [15]:
model_balance = DecisionTreeClassifier(random_state=12345, max_depth=7, class_weight = 'balanced')
model_balance.fit(features_train, target_train)

predict_valid = model_balance.predict(features_valid)
f1_balance = f1_score(target_valid, predict_valid)
print("F1:", f1_balance)

prob_valid1_1 = model_balance.predict_proba(features_valid)
prob_one_valid1_1 = prob_valid1_1[:, 1]
auc_roc1_1 = roc_auc_score(target_valid, prob_one_valid1_1)
print("ROC-AUC:", auc_roc1_1)

F1: 0.5447368421052632
ROC-AUC: 0.8111041342046714


In [16]:
model_balance_1 = RandomForestClassifier(n_estimators=120, random_state=12345, max_depth=10, class_weight = 'balanced')
model_balance_1.fit(features_train, target_train)

predict_valid_1 = model_balance_1.predict(features_valid)
f1_balance_1 = f1_score(target_valid, predict_valid_1)
print("F1:", f1_balance_1)

prob_valid1_2 = model_balance_1.predict_proba(features_valid)
prob_one_valid1_2 = prob_valid1_2[:, 1]
auc_roc1_2 = roc_auc_score(target_valid, prob_one_valid1_2)
print("ROC-AUC:", auc_roc1_2)

F1: 0.5859247135842881
ROC-AUC: 0.8657255595091359


Таким образом, модель "Случайный лес" имеет наибольший показатель f1 и ROC-AUC, в связи с чем, подлежит использованию в дальнейшей работе. При этом достигнуто увеличение метрики f1.


Дополнительно, целесообразно попробовать применить метод увеличения выборки тренировочных данных. При этом метод уменьшения выборки в рассматриваемом случае является менее предпочтительным ввиду относительно малого размера выборки train.

In [17]:
def upsample(features, target, repeat):
    features_zeros = features_train[target_train == 0]
    features_ones = features_train[target_train == 1]
    target_zeros = target_train[target_train == 0]
    target_ones = target_train[target_train == 1]
    
    features_upsampled = pd.concat([features_zeros] + [features_ones] * repeat)
    target_upsampled = pd.concat([target_zeros] + [target_ones] * repeat)
    
    features_upsampled, target_upsampled = shuffle(
        features_upsampled, target_upsampled, random_state=12345)
    
    return features_upsampled, target_upsampled   

features_upsampled, target_upsampled = upsample(features_train, target_train, 4)

Данные подготовлены (увеличены и размещаны) для дальнейшего обучения.

Обучение модели "Решающее дерево" на "увеличенных данных" и расчет метрики f1:

In [18]:
model_balance_2 = DecisionTreeClassifier(random_state=12345, max_depth=7)
model_balance_2.fit(features_upsampled, target_upsampled)

predict_valid_2 = model_balance_2.predict(features_valid)
f1_balance_2 = f1_score(target_valid, predict_valid_2)
print("F1:", f1_balance_2)

prob_valid2_2 = model_balance_2.predict_proba(features_valid)
prob_one_valid2_2 = prob_valid2_2[:, 1]
auc_roc2_2 = roc_auc_score(target_valid, prob_one_valid2_2)
print("ROC-AUC:", auc_roc2_2)

F1: 0.5464788732394367
ROC-AUC: 0.8147088691985085


Обучение модели "Случайный лес" на "увеличенных данных" и расчет метрики f1:

In [19]:
model_balance_3 = RandomForestClassifier(n_estimators=120, random_state=12345, max_depth=10)
model_balance_3.fit(features_upsampled, target_upsampled)

predict_valid_3 = model_balance_3.predict(features_valid)
f1_balance_3 = f1_score(target_valid, predict_valid_3)
print("F1:", f1_balance_3)

prob_valid1_3 = model_balance_3.predict_proba(features_valid)
prob_one_valid1_3 = prob_valid1_3[:, 1]
auc_roc1_3 = roc_auc_score(target_valid, prob_one_valid1_3)
print("ROC-AUC:", auc_roc1_3)

F1: 0.5925925925925927
ROC-AUC: 0.8624257411671072


**Вывод:**

Таким образом, учитывая достаточность размера метрики f1 и ROC-AUC, модель "Случайный лес" является более успешной. При этом необходимо использовать модель уравновешанную по классам. 

## Тестирование модели

Осуществим обучение модели "Случайный лес" на тестовых данных с применением атрибута class_weight:

In [20]:
model = RandomForestClassifier(n_estimators=120, random_state=12345, max_depth=10,  class_weight = 'balanced')
model.fit(features_train, target_train)

predict_test = model.predict(features_test)
f1 = f1_score(target_test, predict_test)
print("F1:", f1)

F1: 0.6126126126126126


In [21]:
prob_valid = model.predict_proba(features_test)
prob_one_valid = prob_valid[:, 1]


auc_roc = roc_auc_score(target_test, prob_one_valid)
print(auc_roc)

0.8763930048199827


Таким образом, достигнуто и превышено значение метрики f1, предусмотренное условиями задачи, что свидетелеьствует об объективности выбранной модели. 

**Общий вывод:**

В ходе выполнения проекта разработана модель, имеющая достаточный уровень метрики f1. При этом в ходе подготовки данных были выявлены и заполнены пропуски в данных, исключены лишние столбцы, которые не имеют значения для обудения данных. Также проведена кодировка данных в целях исключения категориальных данных. Далее данные были разделены на тестовую, обучающую и валидационную выборки. При выполнении проекта были применены и сравнены модели "Случайный лес" и "Решающее дерево", как наиболее перспективные в рассматриваемом случае. Кроме того, выявлены значительный дисбалан данных. 

В заключительной части проекта были применены методы увеличения выборки и уравновешивания по классам (по результатам был выбран уравновешанный "Случайный лес").

Результаты проверки модели "Случайный лес" на тестовой выборке показали, что модель обладает значением f1 0,61, которое превышается значение из условия задачи, что свидетельствует о выполнении поставленной задачи.

Дополнительно установлено, что метрика auc_roc, полученная по результатам проверки на тестовой выборке, значетельно НЕ отличается от значения, полученного на валидационных данных.